In [ ]:
#requirements install all the libraries that is required for the project
!pip install groq
!pip install tensorflow
!pip install	agents
!pip install	streamlit
!pip install	scholarly
!pip install	python-dotenv
!pip install	autogen
!pip install	langchain-community
!pip install	langchain-core
!pip install	streamlit
!pip install	langchain
!pip install	python-dotenv
!pip install	langchain_groq
!pip install	transformers
!pip install	scholarly
!pip install	autogen
!pip install	scikit-learn
!pip install	nltk
!pip install	numpy
!pip install	rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 17.8 MB/s eta 0:00:00
  Created wheel for agents: filename=agents-1.4.0-py3-none-any.whl size=62711 sha256=67da06d85fb29d40ea350bc56ed145339d40fde1c73a31610871bbb809790ba4
  Stored in directory: /root/.cache/pip/wheels/6a/41/e1/4d212ede7c9752d8f7050d78fa3295304bc22b8fb530d9ebe4
Successfully built agents
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  

Code explaination: In the above cell all the libraries that are necessary for running the code are installed

In [ ]:
import requests
import xml.etree.ElementTree as ET
from scholarly import scholarly

class DataLoader:
    def __init__(self):
        print("DataLoader Init")
    def fetch_arxiv_papers(self, query):
        """
            Fetches top 6 research papers from ArXiv based on the user query.
            If <6 papers are found, expands the search using related topics.

            Returns:
                list: A list of dictionaries containing paper details (title, summary, link).
        """

        def search_arxiv(query):
            """Helper function to query ArXiv API."""
            url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results=6"
            response = requests.get(url)
            if response.status_code == 200:
                root = ET.fromstring(response.text)
                return [
                    {
                        "title": entry.find("{http://www.w3.org/2005/Atom}title").text,
                        "summary": entry.find("{http://www.w3.org/2005/Atom}summary").text,
                        "link": entry.find("{http://www.w3.org/2005/Atom}id").text
                    }
                    for entry in root.findall("{http://www.w3.org/2005/Atom}entry")
                ]
            return []

        papers = search_arxiv(query)

        if len(papers) < 6 and self.search_agent:  # If fewer than 6 papers, expand search
            related_topics_response = self.search_agent.generate_reply(
                messages=[{"role": "user", "content": f"Suggest 3 related research topics for '{query}'"}]
            )
            related_topics = related_topics_response.get("content", "").split("\n")

            for topic in related_topics:
                topic = topic.strip()
                if topic and len(papers) < 6:
                    new_papers = search_arxiv(topic)
                    papers.extend(new_papers)
                    papers = papers[:6]  # Ensure max 6 papers

        return papers

    def fetch_google_scholar_papers(self, query):
        """
            Fetches top 6 research papers from Google Scholar.
            Returns:
                list: A list of dictionaries containing paper details (title, summary, link)
        """
        papers = []
        search_results = scholarly.search_pubs(query)

        for i, paper in enumerate(search_results):
            if i >= 6:
                break
            papers.append({
                "title": paper["bib"]["title"],
                "summary": paper["bib"].get("abstract", "No summary available"),
                "link": paper.get("pub_url", "No link available")
            })
        return papers

Code explaination: requests: Used to make HTTP requests to ArXiv's API.
xml.etree.ElementTree (ET): Parses the XML response from ArXiv.
scholarly: A library to interact with Google Scholar.
The DataLoader class is responsible for fetching research papers.
The __init__ method prints "DataLoader Init" when an instance is created.
Constructs an ArXiv API URL for the given query.
Requests up to 6 research papers.
If the request is successful (200 OK):
Converts the response into an XML tree.
Extracts title, summary, and link for each research paper.
Returns a list of dictionaries containing this information.
If fewer than 6 papers are found:
Uses self.search_agent (an external AI model) to suggest 3 related research topics.
Expands the search using these topics.
Searches for additional papers using related topics.
Ensures the total number of papers does not exceed 6.
Iterates through search results (up to 6).
Extracts:
Title from paper["bib"]["title"]
Summary (abstract) if available
Link to the paper (if provided)

In [ ]:
import os
from autogen import AssistantAgent, UserProxyAgent
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class ResearchAgents:
    def __init__(self, api_key, max_turns=3):
        self.groq_api_key = api_key
        self.llm_config = {'config_list': [{'model': 'llama-3.3-70b-versatile', 'api_key': self.groq_api_key, 'api_type': "groq"}]}


        self.feedback_history = []

        # UserProxyAgent - Orchestrates interactions between agents
        self.user_proxy_agent = UserProxyAgent(
            name="user_proxy_agent",
            system_message="You are the controller that helps coordinate different agents for research assistance.",
            llm_config=self.llm_config,
            human_input_mode="ALWAYS",
            code_execution_config=False
        )

        # Summarizer Agent - Summarizes research papers
        self.summarizer_agent = AssistantAgent(
            name="summarizer_agent",
            system_message="Summarize the retrieved research papers and present concise summaries to the user with atlease 200 words, JUST GIVE THE RELEVANT SUMMARIES OF THE RESEARCH PAPER AND NOT YOUR THOUGHT PROCESS.",
            llm_config=self.llm_config,
            human_input_mode="NEVER",
            code_execution_config=False
        )


        #Literature review Agent- Generate literature review
        self.literature_review_agent = AssistantAgent(
            name="literature_review_agent",
            system_message="Generate a structured literature review using the summaries of multiple research papers. Provide an overview of the research trends, findings, gaps, and future directions. ONLY PROVIDE THE REVIEW WITHOUT EXPLANATION OF YOUR THOUGHT PROCESS.",
            llm_config=self.llm_config,
            human_input_mode="NEVER",
            code_execution_config=False
        )


        # Feedback Agent - Get feedback from the user to refine the literature review
        self.feedback_agent = AssistantAgent(
            name="feedback_agent",
            system_message="Ask the user whether the summary looks good or if they want to suggest improvements.",
            llm_config=self.llm_config,
            human_input_mode="NEVER",
            code_execution_config=False
        )

    def summarize_paper(self, paper_summary):
        """Generates a summary of the research paper."""
        summary_response = self.summarizer_agent.generate_reply(
            messages=[{"role": "user", "content": f"Summarize this paper: {paper_summary}"}])
        return summary_response.get("content", "Summarization failed!") if isinstance(summary_response, dict) else str(summary_response)


    def literature_review(self, all_summaries):
        """Generates a structured literature review from multiple research paper summaries."""
        if not all_summaries.strip():
            return "No summaries provided for literature review."
        literature_review_response = self.literature_review_agent.generate_reply(
        messages=[{"role": "user", "content": f"Generate a literature review using these research summaries:\n\n{all_summaries}"}])
        return literature_review_response.get("content", "Literature review generation failed!") if isinstance(literature_review_response, dict) else str(literature_review_response)

    def refine_literature_review_with_feedback(self, literature_review, new_feedback):
        """Refines the literature review based on the feedback provided by the user."""
        refined_message = (f"Here is the original summary of a research paper:\n\n"f"{literature_review}\n\n"f"The user has provided the following feedback:\n{new_feedback}\n\n"f"Please refine the summary to incorporate the feedback while maintaining clarity and accuracy.")
        refined_summary_response = self.feedback_agent.generate_reply(messages=[{"role": "user", "content": refined_message}])
        return refined_summary_response.get("content", "Refinement failed!") if isinstance(refined_summary_response, dict) else str(refined_summary_response)



**Code explaination:**
Class Definition:
**ResearchAgents**
This class orchestrates multiple AI agents to assist in the research process. It involves tasks like summarizing research papers, refining literature review with feedback, and generating a literature review.

Constructor (__init__):
api_key: The API key required to interact with a model.
max_turns: A parameter for controlling how many iterations of interaction will be allowed.
Inside the constructor, the class initializes several agents:

**UserProxyAgent:**

Coordinates interactions between the user and the other agents. The user interacts through this agent.
The human_input_mode="ALWAYS" means it will always expect input from the user.
It helps control and direct the flow of tasks between agents.

**Summarizer Agent:**

The agent responsible for summarizing research papers.
The system_message guides it to focus on summarizing research papers with at least 200 words, only giving relevant content without including its thought process.
human_input_mode="NEVER" means it will not require user input while summarizing.

**Literature Review Agent**:

Takes multiple research paper summaries and generates a structured review.
It organizes the research findings, trends, gaps, and possible future directions, without explaining the thought process behind the review.
Each of these agents is configured to use a particular language model (llama-3.3-70b-versatile from Groq) for generating responses.

**Feedback Agent:**

Collects feedback from the user on the generated literature review.
After summarization, it helps refine the literature review based on the user's suggestions.



Methods:
summarize_paper:
Takes the summary of a research paper (paper_summary), and uses the summarizer agent to generate a detailed summary of it.
It constructs a message for the summarizer agent asking it to summarize the provided content.
The method returns the summarized content or an error message if it fails.

**literature_review:**
Takes the summaries of multiple papers and generates a structured literature review.


**refine_literature_review_with_feedback:**
After generating literature review of a paper, users can provide feedback. This method refines the literature review based on that feedback.
It constructs a message that includes the original literature review and the feedback and asks the feedback agent to refine the literature review.
It returns the refined literature review or an error message if it fails.

In [ ]:
GROQ_API_KEY="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"

In [16]:
import streamlit as st
import os
from dotenv import load_dotenv
#from agents import ResearchAgents
#from data_loader import DataLoader

load_dotenv()

print("ok")

# Streamlit UI Title
st.title("Virtual Literature Review Generator")

# Retrieve the API key from environment variables
groq_api_key = os.getenv("GROQ_API_KEY")

# Check if API key is set, else stop execution
if not groq_api_key:
    st.error("GROQ_API_KEY is missing. Please set it in your environment variables.")
    st.stop()

# Initialize AI Agents for literature review and analysis
agents = ResearchAgents(groq_api_key)

# Initialize DataLoader for fetching research papers
data_loader = DataLoader()

# Input field for the user to enter a research topic
query = st.text_input("Enter a research topic:")

# Variable to store all summaries of paper
if "all_summaries" not in st.session_state:
    st.session_state.all_summaries = []

if "literature_review" not in st.session_state:
    st.session_state.literature_review = ""

# When the user clicks "Search"
if st.button("Search"):
    with st.spinner("Fetching research papers..."):  # Show a loading spinner

        # Fetch research papers from ArXiv
        arxiv_papers = data_loader.fetch_arxiv_papers(query)
        all_papers = arxiv_papers

        # If no papers are found, display an error message
        if not all_papers:
            st.error("Failed to fetch papers. Try again!")
        else:
            processed_papers = []

            # Process each paper: generate small summary
            for paper in all_papers:
                # Fetching the paper summary using user_proxy_agent
                summary_response = agents.user_proxy_agent.initiate_chat(agents.summarizer_agent, message=f"Summarize the following paper: {paper['summary']}", max_turns=1)
                # Accessing the 'content' attribute from the summary_response directly
                if hasattr(summary_response, 'content') and summary_response.content:
                    summary = summary_response.content
                summary = agents.summarize_paper(paper['summary'])  # Generate summary

                # Append each summary to the all_summaries variable
                st.session_state.all_summaries.append(summary)

                processed_papers.append({
                    "title": paper["title"],
                    "link": paper["link"],
                    "summary": summary,
                })

            # Display the processed research papers
            st.write("Top 6 Research Papers based on topic given:")
            for i, paper in enumerate(processed_papers, 1):
                st.write(f"### {i}. {paper['title']}")  # Paper title
                st.write(f"[Read Paper]({paper['link']})")  # Paper link

#code to fetch literature review
if st.button("Fetch Literature Review"):
    combined_summaries = "\n\n".join(st.session_state.all_summaries)
    literature_review_response = agents.user_proxy_agent.initiate_chat(
            agents.literature_review_agent,
            message=f"Generate a literature review using these research summaries:\n\n{combined_summaries}",
            max_turns=1
        )
    if combined_summaries:
        st.session_state.literature_review = agents.literature_review(combined_summaries)
        st.subheader("Literature Review of Paper:")
        st.write(st.session_state.literature_review)

# Get the feedback from user and refine the literature review
st.subheader("Refine the Literature Review Based on Your Feedback:")
feedback = st.text_area("Provide feedback for improvements (optional):")

  # Initialize storage for literature review
if "feedback_history" not in st.session_state:
    st.session_state.feedback_history = []

if st.button("Refine Lierature review"):
    if feedback.strip():  # If user provides feedback
        combined_summaries = "\n\n".join(st.session_state.all_summaries)
        st.session_state.literature_review = agents.literature_review(combined_summaries)
        literature_review=agents.literature_review(combined_summaries)
        st.session_state.feedback_history.append(feedback)

        # Combine past and new feedback
        all_feedback = "\n".join(st.session_state.feedback_history)

        # Refine the literature review based on all feedback
        refined_summary_response = agents.user_proxy_agent.initiate_chat(
            agents.feedback_agent,
            message=f"Refine the following literature review based on user feedback.\n\n"
                    f"Previous Literature Review: {st.session_state.literature_review}\n\n"
                    f"All User Feedback Given:\n{all_feedback}\n\n"
                    f"Provide a revised summary incorporating all feedback.",
            max_turns=2
        )

        # Get the updated literature review
        refined_summary = agents.refine_literature_review_with_feedback(literature_review, all_feedback)

        st.write(f"**Refined Summary:** {refined_summary}")
    else:
        st.warning("No feedback provided. Summary remains the same.")


2025-03-14 17:47:10.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.739 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-14 17:47:10.754 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

ok
DataLoader Init


Environment Setup

1.The code first loads environment variables from a .env file using load_dotenv(). This is crucial for retrieving sensitive data, like API keys.
It checks if the GROQ_API_KEY environment variable exists. If not, it displays an error message using st.error() and halts the execution using st.stop().
2. Streamlit UI
Title: The app displays the title "Virtual Literature Review Generator" using st.title().
API Key Retrieval: The API key required for interacting with external services is fetched from environment variables.
Agents Initialization: The code initializes the ResearchAgents (AI agents for summarization and literature review) and the DataLoader (for fetching research papers).
3. User Input: Research Topic
A text input field is provided (st.text_input("Enter a research topic:")) where the user can enter a research topic.
A check is done for whether summaries and the literature review are available in the session state. If not, they are initialized in st.session_state.
4. Search Papers:
Button "Search": When the user clicks "Search", the app fetches research papers related to the query from ArXiv using the data_loader.fetch_arxiv_papers() function.
The papers are processed one by one. For each paper:
The summarizer_agent generates a summary for the paper using the agents.summarizer_agent.initiate_chat() method.
The summary is stored in st.session_state.all_summaries, and the papers are displayed with their title, link, and summary.
5. Generate Literature Review:
Button "Fetch Literature Review": When the user clicks this button, the app sends the concatenated summaries of all research papers to the literature_review_agent to generate a structured literature review.
If summaries are present in st.session_state.all_summaries, the agents.literature_review() method is called to create the literature review. The result is displayed under "Literature Review of Paper".
6. Feedback and Refining the Review:
Text Area for Feedback: A text area allows the user to provide feedback on the literature review. If the user provides feedback, the app will refine the review based on the feedback.
Button "Refine Literature Review":
When clicked, the app collects all feedback (both past and new) from st.session_state.feedback_history.
It calls the feedback_agent to refine the literature review, incorporating all feedback.
The refined review is then displayed.
7. Storage of Feedback:
Feedback provided by the user is stored in st.session_state.feedback_history, which is a list that accumulates all feedback over time.
When refining the literature review, all feedback is combined and used to revise the review further.
8. Error Handling:
If no papers are found after the "Search" button is pressed, the app displays an error message (st.error("Failed to fetch papers. Try again!")).
If no feedback is provided for refining the literature review, the app will warn the user that the summary remains unchanged using st.warning().


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Example of human-written summary and system-generated summary
human_written_summary = """ Introduction Large Language Models (LLMs) have significantly advanced Multi
Agent Systems (MAS), fostering innovation across diverse domains such as
communication networks, digital twins, security, adaptive learning, problem-solving, and
sustainability. This structured literature review synthesizes recent research on LLM
enhanced MAS, identifying key trends, findings, gaps, and future research directions.
Research Trends The reviewed papers highlight the following research trends:
• Integration of LLMs into MAS: All six papers explore how LLMs can enhance agent
based interactions, decision-making, and automation.
• Domain-Specific Applications: Studies focus on applications such as 6G
communications, digital twins, cybersecurity, adaptive learning, problem-solving,
and sustainability.
• Security and Prompt Injection Attacks: The risks of prompt infection and security
vulnerabilities in multi-agent LLM environments are being investigated.
• Self-Adaptive and Orchestrated Problem Solving: Adaptive multi-agent LLM
systems improve decision-making in dynamic environments.
• Simulation and Parametrization: The role of LLMs in optimizing simulations,
particularly in digital twins and decision support systems, is a recurring theme.
• Interactive and User-Centric Approaches: LLM-powered MAS increasingly focus
on complex problem-solving, decision support, and sustainability goals.
Key Findings
• LLMs Improve Coordination in Multi-Agent Systems: The incorporation of LLMs
enables more efficient communication and task delegation within MAS (Paper 1, 4,
5).
• Enhanced Decision-Making and Adaptability: LLM-based agents facilitate
parameter optimization in simulations and problem-solving (Paper 2, 4, 5, 6).
• Security Vulnerabilities in Multi-Agent Systems: LLM-to-LLM prompt injection
attacks can manipulate agent behavior, highlighting the need for robust security
measures (Paper 3).
• Self-Adaptive Multi-Agent Systems: LLM-based agents can dynamically adjust
strategies to optimize performance and improve problem-solving capabilities
(Paper 4, 5).
• Environmental and Sustainability Applications: The role of LLM-powered MAS in
sustainability decision support, particularly in achieving net-zero emissions, is
gaining attention (Paper 6).
Research Gaps
• Scalability and Computational Efficiency: While promising, LLM-integrated MAS
often require significant computational resources, necessitating research into more
efficient architectures.
• Security and Ethical Considerations: The risks of prompt injection and adversarial
attacks in multi-agent LLM systems need further exploration (Paper 3).
• Explainability and Transparency: The reasoning behind LLM-based agent
decisions remains opaque, raising concerns in critical applications.
• Benchmarking and Standardization: There is a lack of standardized evaluation
metrics to assess the performance of LLM-enhanced MAS across different
domains.
• Human-Agent Interaction: While user-centric approaches are emerging, the
effectiveness of human-agent collaboration in complex real-world settings requires
further empirical validation.
Future Directions
• Optimization and Lightweight Models: Research should explore more
computationally efficient architectures for deploying LLMs in MAS.
• Enhancing Explainability: Developing interpretable LLM-based agents will improve
trust and adoption in critical applications.
• Security Mechanisms: Future studies should focus on robust security frameworks
to mitigate prompt injection attacks and ethical concerns (Paper 3).
• Cross-Domain Applications: Exploring LLM-enhanced MAS beyond the current
domains could unlock new opportunities in healthcare, finance, and autonomous
systems.
• Improved Human-Agent Collaboration: Enhancing user interfaces and feedback
mechanisms can bridge the gap between human users and intelligent agents.
Conclusion The integration of LLMs into MAS is a rapidly evolving research area
with promising applications across multiple domains. While significant advancements
have been made in agent coordination, decision-making, security, and adaptability,
challenges related to scalability, security risks, and explainability remain. Addressing
these gaps will be crucial in unlocking the full potential of LLM-enhanced MAS in real-world
applications."""

system_generated_summary = """ Introduction
Recent advancements in Large Language Models (LLMs) have led to their increasing
integration into Multi-Agent Systems (MASs), enabling enhanced communication, self
adaptation, and cooperation among agents. This literature review aims to provide an
overview of the current research trends, findings, gaps, and future directions in the field of
LLMs and MASs.
Research Trends
The integration of LLMs into MASs has been proposed as a novel approach to improve self
adaptation and communication in autonomic computing. Researchers have explored the
use of LLMs, such as GPT-based technologies, to enhance the expressiveness of
interaction communication in MASs, enabling direct and clear information exchange
between agents. Additionally, LLMs have been leveraged to tackle complex and vague
problems effectively, by decomposing them into tangible sub-problems and assigning
them to specialized LLM agents or non-LLM functions for resolution.
Findings
Studies have demonstrated the effectiveness of LLMs in improving the policy learning
process of multi-agents in cooperative games. The YOLO-MARL framework, which
leverages LLMs for high-level task planning, has been shown to outperform traditional
MARL algorithms. Moreover, researchers have proposed a novel approach to designing
reward functions for Restless Multi-Armed Bandits using LLMs, enabling the allocation of
limited resources among multiple agents based on human preferencesHowever, the
increasing prevalence of multi-agent LLM systems has also raised concerns about their
vulnerability to attacks, such as "Prompt Infection," which can pose severe threats to data
security and system integrity. To mitigate these risks, defense mechanisms, such as "LLM
Tagging," have been proposed to prevent the spread of infection.
Gaps
Despite the promising findings, there are still several gaps in the current research. Firstly,
the scalability and efficiency of LLMs in tackling complex problems need to be further
investigated. Secondly, the development of more advanced defense mechanisms to
prevent attacks on multi-agent LLM systems is crucial. Finally, the study of information
diffusion patterns in LLM communication networks requires further exploration to
understand the dynamics of LLM communication and their potential effects on the quality
and diversity of the information they produce.
Future Directions
Future research should focus on addressing the gaps identified above. Firstly, developing
more scalable and efficient approaches to leveraging LLMs in MASs is essential. Secondly,
investigating more advanced defense mechanisms to prevent attacks on multi-agent LLM
systems is critical. Finally, exploring the dynamics of LLM communication networks and
their potential effects on the quality and diversity of the information they produce can
provide valuable insights into the behavior of LLMs in complex systems.
Furthermore, researchers should investigate the potential applications of LLMs in MASs in
various domains, such as autonomous systems, complex software systems, and public
health. Additionally, the development of more transparent and explainable LLMs is
essential to ensure trust and reliability in their decision-making processes.
In conclusion, the integration of LLMs into MASs has the potential to revolutionize the field
of artificial intelligence, enabling more efficient, adaptable, and cooperative systems.
However, addressing the gaps and challenges identified in this literature review is crucial
to unlocking the full potential of LLMs in MASs.
"""

# ROUGE Score: Use ROUGE for evaluating n-gram recall
def compute_rouge(human_summary, system_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(human_summary, system_summary)
    return scores

# Cosine Similarity: Measure semantic similarity between summaries
def compute_cosine_similarity(human_summary, system_summary):
    vectorizer = CountVectorizer().fit_transform([human_summary, system_summary])
    cosine_sim = cosine_similarity(vectorizer[0], vectorizer[1])
    return cosine_sim[0][0]

# Run the evaluations
rouge_scores = compute_rouge(human_written_summary, system_generated_summary)
cosine_sim = compute_cosine_similarity(human_written_summary, system_generated_summary)

# Output the evaluation metrics
print(f"ROUGE Scores: {rouge_scores}")
print(f"Cosine Similarity: {cosine_sim:.4f}")

ROUGE Scores: {'rouge1': Score(precision=0.5244618395303327, recall=0.47771836007130125, fmeasure=0.5), 'rouge2': Score(precision=0.15294117647058825, recall=0.1392857142857143, fmeasure=0.14579439252336449), 'rougeL': Score(precision=0.2446183953033268, recall=0.22281639928698752, fmeasure=0.23320895522388058)}
Cosine Similarity: 0.6883


Interpreting the Results:
ROUGE-1 is reasonably good, with a precision of about 52.45% and recall of 47.77%, showing that the generated summary captures a decent amount of relevant content.
ROUGE-2 scores are lower, indicating that the generated summary has a smaller overlap at the bigram level. This suggests that the generated text might be missing some higher-order relationships between words.
ROUGE-L is moderate, reflecting that the generated summary has some structure and order that aligns with the reference summary, but it's not perfect.
Cosine Similarity of 0.6883 indicates a moderate similarity, implying the generated summary is somewhat aligned with the reference but could still be improved in terms of exact content and word choice.